# Data Processing/Cleaning

In [1]:
import glob

raw_dir = "../raw/dictionaries/"
out_dir = "../webapp/data/languages/"

In [3]:
# for every folder in raw/ (except en), create a folder in webapp/data/

import os
import glob
import random

# problematic: tlh, vi, sr, ne, el, rw


def dict_to_5words(lang):
    """
    filters a word dictionary to 5-letter words and saves them to a file
    """
    words = []
    with open(raw_dir + lang + "/index.dic", "r") as f:
        i = 0
        for line in f:
            i += 1
            if line[0] == "/":
                continue
            try:
                word = line.split("/")[0]
                word = word.strip().lower()
                forbidden_charset = "0123456789/-"
                forbidden_charset += ""
                if any(c in forbidden_charset for c in word):
                    continue
                if len(word) == 5 and word not in words:
                    words.append(word.lower())
            except Exception as e:
                print(f"Error at line {i}: {e} in {lang}")
                pass

    # if '_characters.txt' file exists in out_dir, load it and filter by it
    try:
        with open(out_dir + lang + "/" + lang + "_characters.txt", "r") as f:
            characters = [line.strip() for line in f]
    except FileNotFoundError:
        characters = []

    # filter words by characters
    if characters != []:
        words = [word for word in words if all([char in characters for char in word])]

    random.seed(42)
    random.shuffle(words)

    with open(out_dir + lang + "/" + lang + "_5words.txt", "w") as f:
        for word in words:
            f.write(word + "\n")

    # make list of all appearing characters
    characters = []
    for word in words:
        for char in word:
            if char not in characters:
                characters.append(char)

    # order characters
    characters.sort()
    # write characters to file if not already there
    if not os.path.exists(out_dir + lang + "/" + lang + "_characters.txt"):
        with open(out_dir + lang + "/" + lang + "_characters.txt", "w") as f:
            for char in characters:
                f.write(char + "\n")

    print(f"{lang} words: {len(words)} characters: {len(characters)}")

    return words

In [4]:
folders = [folder for folder in glob.glob(raw_dir + "*") if "-" not in folder]

for folder in folders:
    lang = folder.split("/")[-1]
    if lang == "en":  # use official word list for english
        continue
    if not glob.glob(out_dir + lang):
        os.mkdir(out_dir + lang)
    words = dict_to_5words(lang)

ie words: 2146 characters: 33
ko words: 8921 characters: 65
ne words: 2196 characters: 48
sl words: 11730 characters: 25
nl words: 7440 characters: 36
lb words: 1751 characters: 29
hu words: 6046 characters: 31
fr words: 4481 characters: 40
oc words: 4203 characters: 38
ia words: 2475 characters: 26
mk words: 5997 characters: 31
ga words: 5081 characters: 23
tr words: 9223 characters: 29
is words: 8284 characters: 32
it words: 2782 characters: 32
ru words: 4687 characters: 32
pt words: 9015 characters: 38
es words: 3601 characters: 33
lv words: 2774 characters: 33
br words: 7142 characters: 28
ca words: 9078 characters: 37
hr words: 3590 characters: 27
et words: 9458 characters: 28
ltg words: 387 characters: 34
sr words: 17967 characters: 30
pl words: 10183 characters: 32
he words: 64539 characters: 27
vi words: 738 characters: 88
fa words: 11252 characters: 37
eu words: 7519 characters: 27
uk words: 9588 characters: 33
ro words: 8617 characters: 30
fur words: 3568 characters: 36
fy wo

In [5]:
# Custom arabic word source
import pandas as pd

df = pd.read_excel(
    raw_dir + "ar/Top-50000-Arabic-Words-Masterlist_ModernStandardArabic.com_.xlsx"
)

,Word Number,Frequency,Arabic,English,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,1,2285403,لا,No,NaN,NaN,NaN,NaN,NaN
1,2,2229495,من,Of,NaN,NaN,NaN,NaN,NaN
2,3,1789391,في,In,NaN,NaN,NaN,NaN,NaN
3,4,1761748,أن,That,NaN,NaN,NaN,NaN,NaN
4,5,1624794,هذا,This is,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4995,4996,2304,احتفظ,Keep a,NaN,NaN,NaN,NaN,NaN
4996,4997,2303,الصخور,Rock,NaN,NaN,NaN,NaN,NaN
4997,4998,2303,أخبروني,They told me,NaN,NaN,NaN,NaN,NaN
4998,4999,2303,اخرجوا,Get out,NaN,NaN,NaN,NaN,NaN


In [40]:
raw_words = df[["Arabic"]].values.tolist()
words = []
for word in raw_words:
    word = word[0]
    # ـ
    forbidden_charset = "abcdefghijklmnopqrstuvwxyz -0123456789"
    try:
        if (
            any(char.isupper() for char in word)
            or any(char in forbidden_charset for char in word)
            or any(char.isdigit() for char in word)
        ):
            continue
        if len(word) == 5 and word not in words:
            words.append(word.lower())
    except Exception as e:
        print(f"{e}")
        pass

# make list of all appearing characters
characters = []
for word in words:
    for char in word:
        if char not in characters:
            characters.append(char)

print(f"Arabic 5 words: {len(words)}")
print(f"Arabic characters: {len(characters)}")
print(" ".join(characters))

'bool' object is not iterable
Arabic 5 words: 14157
Arabic characters: 37
ع ن د م ا ل ذ أ ر ت ق و ي ك ج س ح ة ب ه ش ء ؤ خ ص ظ ث آ ئ ض ط ى غ ز ف إ ـ


In [59]:
# Let's do a more reusable version of all this
import random


def process_wordlist(
    wordlist,
    language_code,
    desired_word_length=5,
    forbidden_charset=None,
    acceptable_charset=None,
):
    """Takes in a wordlist and processes it for wordle consumption"""
    # EDIT: KINDA OUTDATED
    words = []
    for word in wordlist:
        word = word.strip().lower()
        if forbidden_charset:
            if any(char in forbidden_charset for char in word):
                continue
        if acceptable_charset:
            if not any(char in acceptable_charset for char in word):
                continue
        if len(word) == desired_word_length and word not in words:
            words.append(word)

    # mix up the words in case they were in a certain order
    random.seed(42)
    random.shuffle(words)

    characters = []
    for word in words:
        for char in word:
            if char not in characters:
                characters.append(char)

    # write to file
    with open(
        out_dir
        + "languages/"
        + language_code
        + "/"
        + language_code
        + "_"
        + str(desired_word_length)
        + "words.txt",
        "w",
    ) as f:
        for word in words:
            f.write(word + "\n")

    with open(
        out_dir
        + "languages/"
        + language_code
        + "/"
        + language_code
        + "_characters.txt",
        "w",
    ) as f:
        for char in characters:
            f.write(char + "\n")

    print(f"{language_code} words: {len(words)} characters: {len(characters)}")
    print(f"characterset for {language_code}: {' '.join(characters)}")
    return words, characters

In [60]:
words, characters = process_wordlist(
    words, "ar", 5, forbidden_charset="abcdefghijklmnopqrstuvwxyz -0123456789 ـ"
)

ar words: 13882 characters: 36
characterset for ar: و ا ل س ق ب ه م ح أ ج ت ض ي ر ف ة ذ ع ك ن ز ش ص إ د ئ خ ط ظ ء ث غ آ ى ؤ


In [16]:
# let's do Tolkien's Quenya for funsies

url = "https://folk.uib.no/hnohf/quen-eng.htm"
import requests
from bs4 import BeautifulSoup

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# the words are the first bolded text in each top level paragraph
# <p><b>WORD</b>...</p>

# extract words
words = []
for p in soup.find_all("p"):
    try:
        word = p.find("b").text
        word = word.strip().lower()
        if len(word) == 5 and word not in words:
            words.append(word)
    except:
        pass

# shuffle the words
random.seed(42)
random.shuffle(words)

words[:10]

['intye',
 'palme',
 'callo',
 'ormal',
 'nosse',
 'rúnya',
 'pasta',
 'remba',
 'rambe',
 'lauca']

In [18]:
# save to '../webapp/data/languages/qya/qya_5words.txt'
with open(out_dir + "qya/qya_5words.txt", "w") as f:
    for word in words:
        f.write(word + "\n")

# save to '../webapp/data/languages/qya/qya_characters.txt'
characters = []
for word in words:
    for char in word:
        if char not in characters:
            characters.append(char)
characters.sort()
with open(out_dir + "qya/qya_characters.txt", "w") as f:
    for char in characters:
        f.write(char + "\n")

In [26]:
# let's try a better arabic wordlist
# using file: '../raw/ar/hans-wehr.wordlist.txt' and acceptable characters: '../webapp/data/languages/ar/ar_characters.txt'

with open("../raw/ar/hans-wehr.wordlist.txt", "r") as f:
    words = f.read().split("\n")

with open(out_dir + "ar/ar_characters.txt", "r") as f:
    characters = f.read().split("\n")

print(f"Arabic 5 words: {len(words)}")
words_clean = []
for word in words:
    if (
        len(word) == 5
        and word not in words_clean
        and all(char in characters for char in word)
    ):
        words_clean.append(word)

print(len(words_clean))


# shuffle the words
random.seed(42)
random.shuffle(words_clean)

# save to '../webapp/data/languages/ar/ar_5words.txt'
with open(out_dir + "ar/ar_5words.txt", "w") as f:
    for word in words_clean:
        f.write(word + "\n")

Arabic 5 words: 34553
10174


In [25]:
words
len("مستميت")

6

In [2]:
# gather all external wordles from 'https://rwmpelstilzchen.gitlab.io/wordles/'

import requests
from bs4 import BeautifulSoup

response = requests.get("https://rwmpelstilzchen.gitlab.io/wordles/")
soup = BeautifulSoup(response.text, "html.parser")

In [25]:
# find table that has a child caption with text ' Multilingual Wordle-like games; 419 entries'
captions = [
    "🌐 Multilingual Wordle-like games",
    "Domain-specific Wordle-like games",
    "Wordle-like games with a gameplay twist",
]
tables = soup.find_all("table")
wordle_clones = []

for t in tables:
    if t.caption:
        if any(caption in t.caption.text for caption in captions):
            table_body = t.find("tbody")

            # find all direct descendant rows
            rows = table_body.find_all("tr", recursive=False)
            print("Scraping table with caption: " + t.caption.text)
            print("number of rows: " + str(len(rows)))

            current_language_name = ""
            current_language_name_native = ""
            for i, row in enumerate(rows):
                try:
                    wordle_clone = {}
                    cols = row.find_all("td")

                    try:
                        language_name = cols[0].find_all("a")[1].text
                        language_name_native = cols[1].find_all("a")[0].text

                        current_language_name = language_name
                        current_language_name_native = language_name_native
                    except:
                        pass
                    name = cols[2].find_all("a")[0].text
                    url = cols[2].find_all("a")[0].get("href")

                    spans = cols[4].find_all("span")
                    if len(spans) >= 2:
                        note = spans[0].text
                    else:
                        note = ""

                    wordle_clone["language_name"] = current_language_name
                    wordle_clone["language_name_native"] = current_language_name_native
                    wordle_clone["name"] = name
                    wordle_clone["note"] = note
                    wordle_clone["url"] = url

                    wordle_clones.append(wordle_clone)
                except Exception as e:
                    print(f"Error scraping wordle: {e} at row {i}")
                    print(row)
                    print()
        if "Non-linguistic wordles" in t.caption.text:
            table_body = t.find("tbody")

            # find all direct descendant rows
            rows = table_body.find_all("tr", recursive=False)
            print("Scraping table with caption: " + t.caption.text)
            print("number of rows: " + str(len(rows)))

            for i, row in enumerate(rows):
                try:
                    wordle_clone = {}
                    cols = row.find_all("td")

                    name = cols[1].find_all("a")[0].text
                    url = cols[1].find_all("a")[0].get("href")
                    note = cols[3].find_all("span")[0].text

                    wordle_clone["name"] = name
                    wordle_clone["note"] = note
                    wordle_clone["url"] = url

                    wordle_clone["language_name"] = note
                    wordle_clone["language_name_native"] = ""

                    wordle_clones.append(wordle_clone)
                except Exception as e:
                    print(f"Error scraping wordle: {e} at row {i}")
                    print(row)
                    print()

# cols

Scraping table with caption: 🌐 Multilingual Wordle-like games; 419 entries
number of rows: 409
Scraping table with caption: 🔍 Domain-specific Wordle-like games; 105 entries
number of rows: 105
Scraping table with caption: 🃏 Wordle-like games with a gameplay twist; 33 entries
number of rows: 33
Scraping table with caption: 😶 Non-linguistic wordles(s); 32 entries
number of rows: 32


In [3]:
# find table that has a child caption with text ' Multilingual Wordle-like games; 419 entries'
captions = [
    "🌐 Multilingual Wordle-like games",
    "Domain-specific Wordle-like games",
    "Wordle-like games with a gameplay twist",
]
tables = soup.find_all("table")
wordle_clones = []

for t in tables:
    if t.caption:
        if any(caption in t.caption.text for caption in captions):
            table_body = t.find("tbody")

            # find all direct descendant rows
            rows = table_body.find_all("tr", recursive=False)
            print("Scraping table with caption: " + t.caption.text)
            print("number of rows: " + str(len(rows)))

            current_language_name = ""
            current_language_name_native = ""
            for i, row in enumerate(rows):
                try:
                    wordle_clone = {}
                    cols = row.find_all("td")

                    try:
                        language_name = cols[0].find_all("a")[1].text
                        language_name_native = cols[1].find_all("a")[0].text

                        current_language_name = language_name
                        current_language_name_native = language_name_native
                    except:
                        pass
                    name = cols[2].find_all("a")[0].text
                    url = cols[2].find_all("a")[0].get("href")

                    spans = cols[4].find_all("span")
                    if len(spans) >= 2:
                        note = spans[0].text
                    else:
                        note = ""

                    wordle_clone["language_name"] = current_language_name
                    wordle_clone["language_name_native"] = current_language_name_native
                    wordle_clone["name"] = name
                    wordle_clone["note"] = note
                    wordle_clone["url"] = url

                    wordle_clones.append(wordle_clone)
                except Exception as e:
                    print(f"Error scraping wordle: {e} at row {i}")
                    print(row)
                    print()
        if "Non-linguistic wordles" in t.caption.text:
            table_body = t.find("tbody")

            # find all direct descendant rows
            rows = table_body.find_all("tr", recursive=False)
            print("Scraping table with caption: " + t.caption.text)
            print("number of rows: " + str(len(rows)))

            for i, row in enumerate(rows):
                try:
                    wordle_clone = {}
                    cols = row.find_all("td")

                    name = cols[1].find_all("a")[0].text
                    url = cols[1].find_all("a")[0].get("href")
                    note = cols[3].find_all("span")[0].text

                    wordle_clone["name"] = name
                    wordle_clone["note"] = note
                    wordle_clone["url"] = url

                    wordle_clone["language_name"] = note
                    wordle_clone["language_name_native"] = ""

                    wordle_clones.append(wordle_clone)
                except Exception as e:
                    print(f"Error scraping wordle: {e} at row {i}")
                    print(row)
                    print()

# cols

Scraping table with caption: 🌐 Multilingual Wordle-like games; 485 entries
number of rows: 474
Scraping table with caption: 🔍 Domain-specific Wordle-like games; 106 entries
number of rows: 106
Scraping table with caption: 🃏 Wordle-like games with a gameplay twist; 41 entries
number of rows: 41
Scraping table with caption: 😶 Non-linguistic wordles(s); 33 entries
number of rows: 33


In [4]:
# load '../webapp/data/other_wordles.json'
import json

with open("../webapp/data/other_wordles.json", "r") as f:
    other_wordles = json.load(f)

# append wordles to '../webapp/data/other_wordles.json'
for other_wordle in wordle_clones:
    obj_to_add = {
        "name": other_wordle["name"],
        "url": other_wordle["url"],
        "language": f"{other_wordle['language_name']}/{other_wordle['language_name_native']}",
    }
    other_wordles.append(obj_to_add)

# save to '../webapp/data/languages/other_wordles.json'
with open("../webapp/data/other_wordles.json", "w", encoding="utf-8") as f:
    json.dump(other_wordles, f, ensure_ascii=False, indent=4)

In [5]:
wordle_clones

[{'language_name': 'Abkhaz',
  'language_name_native': 'аԥсшәа',
  'name': 'Mecel',
  'note': '',
  'url': 'https://mecel.raxys.app/'},
 {'language_name': 'Afrikaans',
  'language_name_native': 'Afrikaans',
  'name': 'Wortel',
  'note': '',
  'url': 'https://wortel.wrintiewaar.co.za/'},
 {'language_name': 'Afrikaans',
  'language_name_native': 'Afrikaans',
  'name': 'Watse',
  'note': '',
  'url': 'https://watse.klyntji.com/'},
 {'language_name': 'Afrikaans',
  'language_name_native': 'Afrikaans',
  'name': '𒀀𒉿𒀜𒈝',
  'note': '3–5 signs',
  'url': 'https://eggrobin.github.io/awatlum/'},
 {'language_name': 'Albanian',
  'language_name_native': 'gjuha shqipe',
  'name': 'luaj.live',
  'note': '',
  'url': 'https://luaj.live/'},
 {'language_name': 'Armenian (Eastern)',
  'language_name_native': 'արևելա\u200bհայերեն',
  'name': 'ԲԱՌԴԼ',
  'note': '',
  'url': 'https://gagikm.github.io/bardl/'},
 {'language_name': 'Armenian (Eastern)',
  'language_name_native': 'արևելա\u200bհայերեն',
  'name